<a href="https://colab.research.google.com/github/yoly2020/ocr_test_project/blob/main/OCR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###预处理

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

 常数定义

In [ ]:
# CNN constants
CNN_IN_WIDTH=64
CNN_IN_HEIGHT=32

# data augmentation constants
DATA_AUG_POS_SHIFT_MIN=-2
DATA_AUG_POS_SHIFT_MAX=2
DATA_AUG_SCALES=[0,9.1.1]
DATA_AUG_ROT_MIN=-15
DATA_AUG_ROT_MAX=15

# 文件位置相关常量
TRAIN_DIR='flickr_logos_27_dataset'
TRAIN_IMAGE_DIR=os.path.join(TRAIN_DIR,'flickr_logos_27_dataset_images')
CROPPED_AUG_IMAGE_DIR=os.path.join(TRAIN_DIR,'flickr_logos_27_dataset_cropped_augmented_images')
ANNOT_FILE='flickr_logos_27_dataset_training_set_annotation.txt'
NONE_IMAGE_DIR=os.path.join(TRAIN_DIR,'empty_dir')
NUM_OF_NONE_IMAGES=1000

# class constants
CLASS_NAME = ['Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex', 'Ferrari',
    'Ford', 'Google', 'HP', 'Heineken', 'Intel', 'McDonalds', 'Mini', 'Nbc',
    'Nike', 'Pepsi', 'Porsche', 'Puma', 'RedBull', 'Sprite', 'Starbucks',
    'Texaco', 'Unicef', 'Vodafone', 'Yahoo']

加载包

In [ ]:
import numpy as np
from PIL import Image
from collections import defaultdict
from itertools import product
from sklearn.model_selection import train_test_split
import shutil
import re
import glob

In [ ]:
import os
os.chdir('/content/gdrive/My Drive/OCR Project/Logo Project/')

###第一步，image augmentation and cropping

定义所需函数

In [ ]:
# 解析读入的有image标记的txt文件'flickr_logos_27_dataset_training_set_annotation.txt'

def parse_annot(annot):
  fn=annot[0].decode('utf-8') # 解析utf8格式的string
  class_name=annot[1].decode('utf-8')
  train_subset_class=annot[2].decode('utf-8')
  return fn,class_name,train_subset_class


In [ ]:
def get_rect(annot):
  rect=defaultdict(int)
  x1,y1,x2,y2=rect_coord(annot[3:]) # top-left -> bottom-right
  cx,cy,width,height=center_width_height(x1,y1,x2,y2)
  rect['x1']=x1
  rect[]

In [ ]:
def center_width_height(x1,y1,x2,y2):
  cx=x1+(x2-x1)//2 # 向下取整的整除
  cy=y1+(y2-y1)//2
  width=(x2-x1)
  hight=(y2-y1)
  return cx,cy,width,height